In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch09. 1장 vectorEmbeddingModel 성능비교  </font></b>

# 문장 → 임베딩 벡터(1차원 숫자 배열)

- openAI API의 키 OPENAI_API_KEY (text-embedding-3-large)를 .env에 추가
- upstage(한글 임베딩 https://console.upstage.ai/api-keys)의 키 UPSTAGE_API_KEY를 .env에 추가

# 1. 환경변수 load

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 방법

- 1. 유클리드 거리 : 두 벡터간 거리가 가까운지
- 2. 코사인 유사도 : 두 벡터간 방향이 유사한지
- 3. dot product(거의안씀) : 두 벡터간 곱을 사용하여 거리와 방향을 모두 고려
- pinecone(https://www.pinecone.io/)

In [6]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 embedding model 사용

- text-embedding-3-large

In [7]:
from openai import OpenAI
openai_client = OpenAI()

In [8]:
# text-embedding-3-large(과금)

response = openai_client.embeddings.create(
    input="The king is the prince's father.",
    model="text-embedding-3-large"
)

In [10]:
response.usage # prompt_tokens 입력토큰. 현재 출력토큰은 발생 안함

Usage(prompt_tokens=8, total_tokens=8)

In [16]:
king_vector = np.array(response.data[0].embedding) # The king~ 문장을 숫자로 바꾼 vector
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.02941101  0.02669936 -0.01138112 ...  0.00222114  0.01161579
 -0.00230914]


In [17]:
# text-embedding-3-large(과금)

queen_response = openai_client.embeddings.create(
    input="The queen is the prince's mother",
    model="text-embedding-3-large"
)

In [18]:
queen_response.usage # .을 쓰면 1토큰 더 쓰는거니까 쓰지말자

Usage(prompt_tokens=7, total_tokens=7)

In [19]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape) # large모델은 항상 3072개의 차원을 만듬
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [20]:
# king_vector와 queen_vector의 유사도(king 문장의 .을 빼니까 69%까지 오르긴함)

king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print('king 문장과 queen 문장의 유사도 :', king_queen_similarity)

king 문장과 queen 문장의 유사도 : 0.6519308234196786


In [21]:
slave_response = openai_client.embeddings.create(
    input='The slave begs',
    model='text-embedding-3-large'
)

In [23]:
slave_vector = np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

(3072,)
[-0.00868506  0.00686     0.00754387 ... -0.00823628 -0.0112752
  0.00194153]


In [24]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print('king문장과 slave문장의 유사도 :', king_slave_similarity)

king문장과 slave문장의 유사도 : 0.14972818221309522


In [25]:
# 한국어 문장을 벡터로 바꿔도 유사도가 비슷

kor_response = openai_client.embeddings.create(
    input='왕은 왕자의 아버지다',
    model='text-embedding-3-large'
)

In [26]:
kor_king_vector = np.array(kor_response.data[0].embedding)
print(kor_king_vector.shape)
print(kor_king_vector)

(3072,)
[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [27]:
kor_response = openai_client.embeddings.create(
    input='여왕은 왕자의 어머니다',
    model='text-embedding-3-large'
)

In [28]:
kor_queen_vector = np.array(kor_response.data[0].embedding)
print(kor_queen_vector.shape)
print(kor_queen_vector)

(3072,)
[-0.00633463  0.01295298  0.00534164 ... -0.00638844 -0.00050108
  0.0337521 ]


In [29]:
print('왕문장과 여왕문장의 유사도 :', cosine_similarity(kor_king_vector, kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.5744286874279486


In [30]:
response = openai_client.embeddings.create(
    input='노예가 구걸한다',
    model='text-embedding-3-large'
)

In [31]:
kor_slave_vector = np.array(response.data[0].embedding)
print(kor_slave_vector)

[ 0.00521328  0.00312139  0.00267735 ... -0.01136724 -0.00260993
  0.01129488]


In [33]:
print('왕문장과 노예문장의 유사도 :', cosine_similarity(kor_king_vector, kor_slave_vector))

왕문장과 노예문장의 유사도 : 0.13557034352973563


In [34]:
# king문장과 왕문장의 유사도

print('king문장과 왕문장의 유사도 :', cosine_similarity(king_vector, kor_king_vector))

king문장과 왕문장의 유사도 : 0.6446928580128927


# 4. upstage의 embedding model 사용

- 한국어에서는 openai보다 성능이 좋다

In [35]:
upstage_client = OpenAI(
    api_key=upstage_key,
    base_url="https://api.upstage.ai/v1"
)

In [36]:
response = upstage_client.embeddings.create(
    input="The king is the prince's father",
    model="embedding-query"
)

In [39]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape) # openai보다 생성 수가 많아서 조금 더 느리고 조금 더 잘 맞춤
print(up_king_vector)

(4096,)
[-0.00077958 -0.03257429 -0.00700347 ... -0.00855075  0.00813679
  0.03846481]


In [40]:
response = upstage_client.embeddings.create(
    input="The queen is the prince's mother",
    model="embedding-query"
)

up_queen_vector = np.array(response.data[0].embedding)

In [41]:
print('king문장과 queen문장의 upstage 유사도 :', cosine_similarity(up_king_vector, up_queen_vector))

king문장과 queen문장의 upstage 유사도 : 0.6740461219025426


In [42]:
response = upstage_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="embedding-query"
)

In [43]:
up_kor_king_vector = np.array(response.data[0].embedding)
print(up_kor_king_vector.shape)

(4096,)


In [44]:
response = upstage_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="embedding-query"
)

In [45]:
up_kor_queen_vector = np.array(response.data[0].embedding)
print(up_kor_queen_vector.shape)

(4096,)


In [46]:
print('왕문장과 여왕문장의 upstage 유사도 :', cosine_similarity(up_kor_king_vector, up_kor_queen_vector))

왕문장과 여왕문장의 upstage 유사도 : 0.6502865998932696


In [47]:
print('king문장과 왕문장의 유사도 :', cosine_similarity(up_king_vector, up_kor_king_vector))

king문장과 왕문장의 유사도 : 0.8411266193392042


In [48]:
print('queen문장과 영왕문장의 유사도 :', cosine_similarity(up_queen_vector, up_kor_queen_vector))

queen문장과 영왕문장의 유사도 : 0.8005423469553388
